# Data Scraping

In [1]:
#Importing libraries needed

import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import warnings
import time
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.simplefilter(action = 'ignore', category=SettingWithCopyWarning)

The data we will use correspond to all the players who got at least one MVP vote from 1980 (beginning of the 3-point era) till 2022. All original data come from basketball-reference.com. We begin by getting the data included in the MVP tables.

In [2]:
#Setting the year marks we need (useful for loops later)

first = 1980
last = 2023
years = [i for i in range(first, last)]
seasons = len(years)


#Parsing the tables with the MVP candidates for every year and storing the dfs in a list

mvp_all = []
columns = ["Rank", 'Player',  'Age', 'Tm', 'First', 'Pts Won',
           'Pts Max', 'Share', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL',
           'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48']

for year in years:
    mvp_url = "https://www.basketball-reference.com/awards/awards_{}.html".format(year)
    all_awards_year = pd.read_html(mvp_url)
    time.sleep(5)
    mvp_year = all_awards_year[0]
    mvp_year.columns = columns
    mvp_year["Year"] = year
    mvp_all.append(mvp_year) 

Now we want to get the full data corespinding to these players. In the remaining of this notebook we extract tables corresponding to statistics per game and per 100 possessions as well as advanced statistics. Most features in per game and per 100 possessions correspond to the same statistic (points, rebounds, assists etc) just measured at a different rate. However there are two features in the 'per 100 possessions' tables that don't have a correspondent one (offensive and defensive rating) and are very good indicators of the performance of a player. Thus we need to extract these two and add to the per game stats.

In [3]:
mvp_per_game_stats = []
mvp_per_possession_stats = []
mvp_advanced_stats = []


    
for year in years:
    
    names = mvp_all[year-first]["Player"].values
    
    per_game_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    per_possession_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_poss.html".format(year)
    advanced_url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)
    
    per_game_tab = pd.read_html(per_game_url)
    per_possession_tab = pd.read_html(per_possession_url)
    advanced_tab = pd.read_html(advanced_url)
    time.sleep(5)

    
    per_game_tab[0]["Player"] = per_game_tab[0]["Player"].str.replace("*","")
    per_possession_tab[0]["Player"] = per_possession_tab[0]["Player"].str.replace("*","")
    advanced_tab[0]["Player"] = advanced_tab[0]["Player"].str.replace("*","")

    
    per_game_tab_mvp = per_game_tab[0].loc[per_game_tab[0]["Player"].isin(names)]
    per_possession_tab_mvp = per_possession_tab[0].loc[per_possession_tab[0]["Player"].isin(names)]
    advanced_tab_mvp = advanced_tab[0].loc[advanced_tab[0]["Player"].isin(names)]
    
    per_game_tab_mvp["Year"] = year
    per_possession_tab_mvp["Year"] = year
    advanced_tab_mvp["Year"] = year
    
    mvp_per_game_stats.append(per_game_tab_mvp)
    mvp_per_possession_stats.append(per_possession_tab_mvp)
    mvp_advanced_stats.append(advanced_tab_mvp)

Now that we have the tables, we change all numerical features to float (except two that are integers) and remove two columns we will not need.

In [4]:
per_game_to_float_cols = ['MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

per_possession_to_float_cols = ['MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'ORtg', 'DRtg']

advanced_to_float_cols = ['MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

to_integer_cols = ['Age', 'G']

for i in range(seasons):
    
    mvp_per_game_stats[i].drop(['Rk', 'GS'], axis=1, inplace=True)
    mvp_per_possession_stats[i].drop(['Rk', 'GS'], axis=1, inplace=True)
    mvp_advanced_stats[i].drop('Rk', axis=1, inplace=True)
    
    
    mvp_per_game_stats[i].drop_duplicates(subset=['Player'], keep = 'first', inplace=True)
    mvp_per_possession_stats[i].drop_duplicates(subset=['Player'], keep = 'first', inplace=True)
    mvp_advanced_stats[i].drop_duplicates(subset=['Player'], keep = 'first', inplace=True)
    

    mvp_per_game_stats[i][per_game_to_float_cols] = mvp_per_game_stats[i][per_game_to_float_cols].astype(float)
    mvp_per_game_stats[i][to_integer_cols] = mvp_per_game_stats[i][to_integer_cols].astype(int)
    
    mvp_per_possession_stats[i][per_possession_to_float_cols] = mvp_per_possession_stats[i][per_possession_to_float_cols].astype(float)
    mvp_per_possession_stats[i][to_integer_cols] = mvp_per_possession_stats[i][to_integer_cols].astype(int)
    
    mvp_advanced_stats[i][advanced_to_float_cols] = mvp_advanced_stats[i][advanced_to_float_cols].astype(float)
    mvp_advanced_stats[i][to_integer_cols] = mvp_advanced_stats[i][to_integer_cols].astype(int)

A quick check to see if we have all the columns we need.

In [5]:
print(mvp_per_game_stats[29].columns)
print(mvp_per_possession_stats[29].columns)
print(mvp_advanced_stats[29].columns)

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')
Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Unnamed: 29', 'ORtg',
       'DRtg', 'Year'],
      dtype='object')
Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'Year'],
      dtype='object')


Now it is time to combine the per game stats with the advanced ones in one dataframe and stack all different years in one final table. Earlier we added a column corresponding to the year so we can still separate rows even after stacking all seasons together.

In [6]:
all_per_game = pd.concat(mvp_per_game_stats, ignore_index=True)
all_per_possession = pd.concat(mvp_per_possession_stats, ignore_index=True)
all_per_possession.drop('Unnamed: 29', axis=1, inplace=True)
all_advanced = pd.concat(mvp_advanced_stats, ignore_index=True)
all_advanced.drop(columns=['Unnamed: 19', 'Unnamed: 24'], axis=1, inplace=True)

In [7]:
mvp_full = pd.concat(mvp_all, ignore_index=True)

pure_mvp_full = mvp_full[['Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won', 'Pts Max', 'Share', 'WS', 'WS/48']]

Next we isolate features not included in the initial MVP tables to be added. We use columns Player, Team and Year for merging.

In [8]:
advanced_to_merge = all_advanced.drop(['Age', 'Pos', 'G', 'MP', 'WS', 'WS/48'], axis=1)
per_game_to_merge = all_per_game[["Player", "Tm", "ORB", "DRB", "TOV", "PF", "Year"]]

ratings = all_per_possession[["Player", "Tm", "ORtg", "DRtg", "Year"]]

added_stats_df = pd.merge(pd.merge(per_game_to_merge, advanced_to_merge, on = ["Player", "Tm", "Year"], how = "left"), ratings, on = ["Player", "Tm", "Year"], how = 'left')

mvp_all_stats = pd.merge(mvp_full, added_stats_df, on = ["Player", "Tm", "Year"], how = 'left')


Now we have a small issue we need to address. There are a handful of players that despite being good enough to get MVP votes, they got traded during the season and in result played for two (or more) different teams. These players have multiple entries in the dataset, one row for each team they played plus one corresponding to their full statistics during the season. This row is identified by the Team column that has "TOT" for total in it. Apart from one case however these players were at the bottom of the MVP list and thus can be removed without affecting the dataset. There is only one case where the player got traded and still got too many votes to be easily deleted. This is Chauncey Billups from 2009. Since he only played 2 games with his first team we can safely assign him fully to his second team which was the Denver Nuggets.

In [9]:
mvp_all_stats[(mvp_all_stats["Player"] == "Chauncey Billups") & 
              (mvp_all_stats["Tm"] == "TOT")] = mvp_all_stats[
                                                    (mvp_all_stats["Player"] == "Chauncey Billups") &
                                                    (mvp_all_stats["Tm"] == "TOT")
                                                    ].replace("TOT", "DEN")



Next we want to add a single team feature in our data. This is number of wins or their winning percentage. For now we will add both columns to have available and see later which to keep. For that we need to extract the standings for each year. In the team standings however the name of the team is fully spelled out while in the player statistics only an abbreviation is used. To make the match we create the following dictionary matching full names to abbreviations.

In [10]:
#A dictionary with NBA teams full names and abbreviations, so that we can extract and match wins per season for players


team_dict = {"Boston Celtics": "BOS", "Brooklyn Nets": "BKN", "New York Knicks": "NYK",
             "Philadelphia 76ers": "PHI", "Toronto Raptors": "TOR", "Chicago Bulls": "CHI",
             "Cleveland Cavaliers": "CLE", "Detroit Pistons": "DET", "Indiana Pacers": "IND",
             "Milwaukee Bucks": "MIL", "Atlanta Hawks": "ATL", "Charlotte Hornets": "CHH",
             "Miami Heat": "MIA", "Orlando Magic": "ORL", "Washington Wizards": "WAS",
             "Denver Nuggets": "DEN", "Minnesota Timberwolves": "MIN", "Oklahoma City Thunder": "OKC",
             "Portland Trail Blazers": "POR", "Utah Jazz": "UTA","Golden State Warriors": "GSW",
             "Los Angeles Clippers": "LAC", "Los Angeles Lakers": "LAL", "Phoenix Suns": "PHO",
             "Sacramento Kings": "SAC", "Dallas Mavericks": "DAL", "Houston Rockets": "HOU",
             "Memphis Grizzlies": "MEM", "New Orleans Pelicans": "NOP", "San Antonio Spurs": "SAS",
             "Seattle SuperSonics": "SEA", "Kansas City Kings": "KCK","New Jersey Nets": "NJN",
             "Washington Bullets": "WSB", "Charlotte Bobcats": "CHA", "New Orleans Hornets": "NOH",
             "San Diego Clippers": "SDC", "New Orleans/Oklahoma City Hornets": "NOK",
             "Vancouver Grizzlies": "VAN"
            }



Notice that NBA splits its standings into two conferences, East and West. Thus we need to scrape two tables per year. Next step is to merge these two tables per year into one and then standing of all yers together into one table. Then the standings dataframe is merged on the player_stats dataframe on Team and Year.

In [11]:
#The basketball-reference websites have different structure from 2016 so we need 
#to slightly change the code. Hence a separate loop. 

west_stands = []
east_stands = []
for i in range(1980,2016):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html".format(i)
    tables = pd.read_html(url)
    time.sleep(5)
    east = tables[0][~tables[0].W.str.contains("|".join("Division"))]
    west = tables[1][~tables[1].W.str.contains("|".join("Division"))]
    east["Eastern Conference"] = east["Eastern Conference"].str.replace("*","")
    west["Western Conference"] = west["Western Conference"].str.replace("*","")
    east = east.replace({"Eastern Conference":team_dict})
    west = west.replace({"Western Conference":team_dict})
    east = east[["Eastern Conference", "W", "W/L%"]]
    west = west[["Western Conference", "W", "W/L%"]]
    east = east.rename(columns={'Eastern Conference': 'Tm'})
    west = west.rename(columns={'Western Conference': 'Tm'})
    east_stands.append(east)
    west_stands.append(west)
    
for i in range(2016, 2023):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html".format(i)
    tables = pd.read_html(url)
    time.sleep(5)
    east = tables[2][~tables[2].W.str.contains("|".join("Division"))]
    west = tables[3][~tables[3].W.str.contains("|".join("Division"))]
    east["Eastern Conference"] = east["Eastern Conference"].str.replace("*","")
    west["Western Conference"] = west["Western Conference"].str.replace("*","")
    east = east.replace({"Eastern Conference":team_dict})
    west = west.replace({"Western Conference":team_dict})
    east = east[["Eastern Conference", "W", "W/L%"]]
    west = west[["Western Conference", "W", "W/L%"]]
    east = east.rename(columns={'Eastern Conference': 'Tm'})
    west = west.rename(columns={'Western Conference': 'Tm'})
    east_stands.append(east)
    west_stands.append(west)

In [12]:
#Unite standings for eastern and western conference per year and add a Year column to match with Players data.


all_standings_per_year = []
for i in range(len(west_stands)):
    east_stands[i][["W", "W/L%"]] = east_stands[i][["W", "W/L%"]].astype({'W':int, 'W/L%':float})
    west_stands[i][["W", "W/L%"]] = west_stands[i][["W", "W/L%"]].astype({'W':int, 'W/L%':float})
    east_stands[i]["Year"] = i + 1980
    west_stands[i]["Year"] = i + 1980
    standings_per_year = pd.concat([east_stands[i], west_stands[i]], ignore_index=True)
    all_standings_per_year.append(standings_per_year)
    
full_standings = pd.concat(all_standings_per_year, ignore_index=True)
mvp_df = pd.merge(mvp_all_stats, full_standings, on = ['Tm', 'Year'], how = 'left')

One final thing to do. Many players (especially in the older years) did not attempt any 3-point shots. Thus their percentage is set to NaN. We will change it to zero for now to avoid problems when we build our models. As a feature it should be fairly weak. If it turns out to be more important than thought we can change it to something more meaningful.

In [13]:
mvp_df['3P%'] = mvp_df['3P%'].fillna(0)
mvp_df.dropna(inplace = True)
mvp_df.reset_index(drop = True, inplace = True)

Finally we save the table into a csv file to use in our analysis.

In [152]:
#Save data to a csv file

mvp_df.to_csv('data/MVP_per_game.csv', index=False)